In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
from collections import Counter
import math
import numbers
from itertools import combinations

##########################################################################################
##               Data Preprocessing  load patient's info
##########################################################################################
def patient_info_df(age_threshold):
    age_df = pd.read_csv('../age/age.csv')
    age_df.Age = age_df.Age.apply(lambda x: 'Young' if x <= age_threshold else 'Old')

    gender_df = pd.read_csv('../gender/gender.csv')
    gender_df.Gender = gender_df.Gender.astype(str).apply(lambda x: x.strip())

    smoke_df = pd.read_csv('../smoker/smoker.csv')
    tmp_df = pd.merge(age_df, gender_df, on='Patient_id', how='outer')
    tmp_df = pd.merge(tmp_df, smoke_df, on='Patient_id', how='outer')
    # assert len(age_df) == len(gender_df) == len(smoke_df) == len(tmp_df)
    return tmp_df

def patient_mutation():
    return pd.read_csv('../mutation/mutation_translate.csv')

def patient_relative():
    relative_df = pd.read_csv('../family/family.csv')
    ehrs = set(relative_df.loc[(relative_df.hasCancerFamily == 0.0)].Patient_id)    # patient who has no cancered family
    
    relative_df.Family.mask(relative_df.Patient_id.isin(ehrs), 'No')
    relative_df.FamilyGender.mask(relative_df.Patient_id.isin(ehrs), 'No')
    relative_df.FamilyDegree.mask(relative_df.Patient_id.isin(ehrs), 'No')
    relative_df.FamilyCancer.mask(relative_df.Patient_id.isin(ehrs), 'No')
    relative_df.FamilyCancer = relative_df.FamilyCancer.fillna('UNK').astype(str).apply(lambda x: x.strip()).replace(to_replace={'UNK': np.nan})
    
    relative_df = relative_df.replace(to_replace={"UNK":np.nan, '-': np.nan})   # UNK family to NULL
    
    return relative_df.drop(columns=['hasCancerFamily'])
    

gender_age_smoker_df = patient_info_df(age_threshold=50)
mutation_df = patient_mutation()
family_df = patient_relative()

def reload_data(age_threshold=50):
    global gender_age_smoker_df, mutation_df, family_df
    gender_age_smoker_df = patient_info_df(age_threshold)
    mutation_df = patient_mutation()
    family_df = patient_relative()

In [4]:
def sub_pop(gender=None, age=None, biomarker=None, smoking=None):
    df=copy.deepcopy(gender_age_smoker_df)
    if gender:
        if gender not in df.Gender.unique():
            raise Exception("the value of Gender {gender} is not acceptable".format(gender))
        df = df.loc[(df.Gender == gender)]
    print(gender, "Patient number:", len(df.Patient_id.unique()))
    if age:
        if age not in df.Age.unique():
            raise Exception("the value of Age {age} is not acceptable".format(age))
        df = df.loc[(df.Age == age)]
    print(age, gender, "Patient number:", len(df.Patient_id.unique()))
    if smoking:
        if smoking not in df.Smoker.unique():
            raise Exception("the value of smoker {smoking} is not acceptable".format(smoking))
        df = df.loc[(df.Smoker == smoking)]
    # print(age, gender, smoking, "Patient number:", len(df.Patient_id.unique()))
    
    if biomarker:
        if biomarker not in mutation_df.Biomarker.unique():
            raise Exception("the value of biomarker {biomarker} is not acceptable")
        p_ids = set(mutation_df.loc[(mutation_df.Biomarker == biomarker)].Patient_id)
        df = df.loc[(df.Patient_id.isin(p_ids))]
    print(age, gender,biomarker, "Patient number:", len(df.Patient_id.unique()))
    
    return set(df.Patient_id)


def get_normalized_family_cancer(p_ids):
    '''
    biomarker not null,
    smoker not null,
    age not null,
    gender not null,
    family not null,
    '''
    
    # allow biomarker, gender, smoker, age, family, familycancer shouldn't be null
    p_ids = p_ids.intersection(set(mutation_df.loc[mutation_df.Biomarker.notnull()].Patient_id))
    p_ids = p_ids.intersection(set(gender_age_smoker_df.loc[(gender_age_smoker_df.Smoker.notnull()) & 
    (gender_age_smoker_df.Gender.notnull()) & 
    (gender_age_smoker_df.Age.notnull())].Patient_id))
    df1 = family_df.loc[(family_df.Family.notnull()) & (family_df.FamilyCancer !='Others') & family_df.FamilyCancer.notnull() & (family_df.Patient_id.isin(p_ids))][['Patient_id', 'Family', 'FamilyCancer']]
    
    print("filter(family not null, cancer not null), Patient number:", len(df1.Patient_id.unique()))

    # normalized familiar cancer frequency
    df1['family_cancer'] = df1['Family'] + df1['FamilyCancer']
    df1['family_cancer_num'] = df1.groupby('Patient_id')['family_cancer'].transform('count')
    df1['family_cancer_num'].mask(df1.Family == 'No', 0)    # no family, the number should be 0
    
    df1.sort_values(by=['family_cancer_num']).to_csv('cancer_family_num.csv')
    
    df1 = df1.drop(columns=['Family', 'FamilyCancer', 'family_cancer'])
    df1 = df1.drop_duplicates()

    df1['Normalized Familial Cancer Frequency'] = df1['family_cancer_num'] / df1['family_cancer_num'].max() if df1['family_cancer_num'].max() != 0 else 0
    assert len(df1) == len(df1.Patient_id)
    # df1.rename(columns={'family_cancer_num': 'Normalized Familial Cancer Frequency'}, inplace=True)
    
    # include the biomarker and smoker type of patients for box plot
    df1 = pd.merge(left=df1, right=gender_age_smoker_df[['Patient_id', 'Smoker']], on='Patient_id', how='inner')
    
    assert len(gender_age_smoker_df) == len(gender_age_smoker_df.Patient_id.unique())
    
    print("number of Never-Smoker:", len(df1.loc[(df1.Smoker=='Never-Smoker')].Patient_id.unique()))
    # df1.loc[(df1.Smoker=='Never-Smoker')].sort_values(by=['family_cancer_num']).to_csv('neversmoker.csv')
    
    print("number of Former-Smoker:", len(df1.loc[(df1.Smoker=='Former-Smoker')].Patient_id.unique()))
    # df1.loc[(df1.Smoker=='Former-Smoker')].sort_values(by=['family_cancer_num']).to_csv('runing_example_tmp/formersmoker.csv')
    
    print("number of Current-Smoker:", len(df1.loc[(df1.Smoker=='Current-Smoker')].Patient_id.unique()))
    
    df1.loc[(df1.Smoker.isin(['Current-Smoker','Former-Smoker','Never-Smoker']))].sort_values(by=['Smoker','family_cancer_num']).to_csv('python.csv',index=False)
    
    df1 = pd.merge(left=df1, right=mutation_df[['Patient_id', 'Biomarker']], on='Patient_id', how='inner')
    df1 = df1.drop_duplicates()
    return df1


def Jaccard_df(df1, jacard_choice=0):
    if jacard_choice == 0:  # family type + cancer type
        df1 = df1.loc[(df1.FamilyCancer.notnull()) & (df1.Family.notnull()) & (df1.Family.notnull()) & (df1.FamilyCancer.notnull())]
        df1['jacard_attribute'] = df1['Family']+ df1['FamilyCancer']
    elif jacard_choice == 1:    # fanily type
        df1 = df1.loc[(df1.Family.notnull()) & (df1.Family.notnull())]
        df1['jacard_attribute'] = df1['Family']
    elif jacard_choice == 2:    # cancer type
        df1 = df1.loc[(df1.FamilyCancer.notnull()) & (df1.FamilyCancer.notnull())]
        df1['jacard_attribute'] = df1['FamilyCancer']
    elif jacard_choice == 3:    # degree
        df1 = df1.loc[(df1.FamilyDegree.notnull()) & (df1.FamilyDegree.notnull())]
        df1['jacard_attribute'] = df1['FamilyDegree']
    elif jacard_choice == 4:    # degree + cancer type
        df1 = df1.loc[(df1.FamilyDegree.notnull()) & (df1.FamilyCancer.notnull()) & (df1.FamilyCancer.notnull()) & (df1.FamilyDegree.notnull())]
        df1['jacard_attribute'] = df1['FamilyDegree'] + df1['FamilyCancer']
    elif jacard_choice == 5:    # gender
        df1 = df1.loc[(df1.FamilyGender.notnull()) & (df1.FamilyGender.notnull())]
        df1['jacard_attribute'] = df1['FamilyGender']
    elif jacard_choice == 6:    # gender + cancer
        df1 = df1.loc[(df1.FamilyGender.notnull()) & (df1.FamilyCancer.notnull()) & (df1.FamilyCancer.notnull()) & (df1.FamilyGender.notnull())]
        df1['jacard_attribute'] = df1['FamilyGender'] + df1['FamilyCancer']
    else:
        raise Exception("No such a choice={choice}")
    
    pop_ids = set(df1.Patient_id.unique())
    family_cancer_series = df1.groupby('Patient_id').apply(lambda x: x.jacard_attribute.tolist())
    
    result_df = {'Familial Cancer Connectedness':[]}
    
    for p_id1, p_id2 in list(combinations(df1.Patient_id.unique().tolist(), 2)):
        list1 = family_cancer_series[p_id1]
        list2 = family_cancer_series[p_id2]
        s1 = set(list1)
        s2 = set(list2)

        if len(s1) == 0 or len(s2) == 0:
            result_df['Familial Cancer Connectedness'].append(0)
            continue
        
        if 'No' in s1 or 'No' in s2 or 'NoNo' in s1 or 'NoNo' in s2:    # one of patient has no family with cancer
            result_df['Familial Cancer Connectedness'].append(0)
            continue
        
        jaccard_val = len(s1.intersection(s2)) / len(s1.union(s2)) 
        
        if np.isnan(jaccard_val):
            raise Exception("jaccard_val is np.nan; ", "s1:", s1, "s2", s2)
        result_df['Familial Cancer Connectedness'].append(jaccard_val)
    
    result_df = pd.DataFrame(result_df)
    # print( gender, age, smoker, biomarker, "\t\tpop size", len(df1.Patient_id.unique()), '\t\tavg-Jaccard:', '{:.4f}'.format(result_df['Familial Cancer Connectedness'].mean()))
    return result_df, pop_ids


def box_plot(df, y_var, x_var, hue_var, title, fn):
    import seaborn as sns
    import matplotlib.pyplot as plt

    sns.set_theme(style="ticks", palette="husl")
    sns.set_palette("pastel")
    
    # Draw a nested boxplot to show bills by day and time
    ax = sns.boxplot(x=x_var, y=y_var,
                hue=hue_var, 
                hue_order=["Never-Smoker", "Former-Smoker", 'Current-Smoker'],
                order=sorted(df[x_var].unique().tolist()),
                palette={"Never-Smoker": "red", "Former-Smoker": "green", 'Current-Smoker': "blue"},
                data=df)
    ax.set(title = title)
    
    plt.subplots_adjust(bottom=0.25)   # 图片边缘距离
    if max([len(e) for e in df[x_var].unique()]) > 7:
        plt.setp(ax.get_xticklabels(), rotation=25)
    
    plt.ylim([0, 1+0.13])
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.0),
          ncol=3, fancybox=True, framealpha=0.5, shadow=False, fontsize=9)
    
    ax.get_figure().savefig(fn)
    plt.clf()
    plt.show()

# Running Example

In [6]:
p_ids = sub_pop(gender='Female', age='Young', biomarker='ALK')
df = get_normalized_family_cancer(p_ids=p_ids)
df = df.loc[(df.Biomarker == 'ALK')]
box_plot(df, y_var='Normalized Familial Cancer Frequency', x_var='Biomarker', hue_var='Smoker', title='running', fn='tmp.png')

Female Patient number: 3957
Young Female Patient number: 415
Young Female ALK Patient number: 46
filter(family not null, cancer not null), Patient number: 27
number of Never-Smoker: 17
number of Former-Smoker: 7
number of Current-Smoker: 3


<Figure size 432x288 with 0 Axes>

In [7]:
SELECT count(DISTINCT EHR) FROM `patient` where gender=1
3957

SELECT count(DISTINCT t1.EHR) FROM `patient` t1, `PatientAge` t2 where t1.EHR = t2.EHR and t1.gender=1 and t2.Age <= 50
415

SELECT count(DISTINCT t1.EHR) FROM `patient` t1, `PatientAge` t2, `alk_mutation` t3
where t1.EHR = t2.EHR and t1.EHR = t3.EHR and
t1.gender=1 and t2.Age <= 50 and (`result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0')
46

SELECT count(DISTINCT t1.EHR) FROM `patient` t1, `PatientAge` t2, `alk_mutation` t3, `family_antecedents_treatment_line` t4
where t1.EHR = t2.EHR and t1.EHR = t3.EHR and t1.EHR = t4.EHR and
t1.gender=1 and t2.Age <= 50 and (`result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') and t4.cancer_type not in (-1,888) and t4.family_member is not NULL
27

SELECT count(DISTINCT t1.EHR) FROM `patient` t1, `PatientAge` t2, `alk_mutation` t3, `family_antecedents_treatment_line` t4,  `smoker_information` t5
where t1.EHR = t2.EHR and t1.EHR = t3.EHR and t1.EHR = t4.EHR and t1.EHR = t5.EHR and
t1.gender=1 and t2.Age <= 50 and (`result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') and t4.cancer_type not in (-1,888) and t4.family_member is not NULL and  t5.smoking_habit=0
17

SELECT count(DISTINCT t1.EHR) FROM `patient` t1, `PatientAge` t2, `alk_mutation` t3, `family_antecedents_treatment_line` t4,  `smoker_information` t5
where t1.EHR = t2.EHR and t1.EHR = t3.EHR and t1.EHR = t4.EHR and t1.EHR = t5.EHR and
t1.gender=1 and t2.Age <= 50 and (`result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') and t4.cancer_type not in (-1,888) and t4.family_member is not NULL and  t5.smoking_habit=1
7

SELECT count(DISTINCT t1.EHR) FROM `patient` t1, `PatientAge` t2, `alk_mutation` t3, `family_antecedents_treatment_line` t4,  `smoker_information` t5
where t1.EHR = t2.EHR and t1.EHR = t3.EHR and t1.EHR = t4.EHR and t1.EHR = t5.EHR and
t1.gender=1 and t2.Age <= 50 and (`result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') and t4.cancer_type not in (-1,888) and t4.family_member is not NULL and  t5.smoking_habit=2
3

select t.EHR, count(concat(t.family, CONVERT(t.cancer,char))) as family_cancer_count FROM 
-- one patient has multiple treatment_line in t4 (using distinct)
(SELECT DISTINCT t1.EHR as EHR, t2.Age, t5.gender, t4.cancer_type as cancer, t6.smoking_habit as smoker, t7.familyMember_Translated as family
FROM `alk_mutation` t1, `PatientAge` t2, `family_antecedents` t3,  `family_antecedents_treatment_line` t4, `patient` t5, `smoker_information` t6, `familyRelationshipsENG` t7
where t1.EHR = t2.EHR and t1.EHR = t3.EHR and t1.EHR = t4.ehr and t1.EHR = t5.EHR and t1.EHR = t6.EHR and t4.family_member = t7.familyMember
and t2.Age <= 50 and t5.gender = 1 and (`result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') and t3.has_family_cancer_antecedents != -1 and t4.cancer_type not in (-1,888) and t6.smoking_habit not in (3,-1)) t
where t.smoker = 0  
group by t.EHR
order by family_cancer_count

-- smoker = : never smoker, 1: former smoker, 2: previous smoker

-- 17 Never-Smoker (smoking_habit = 0)
1503660 1
1508901 1
203512  1
203552  1
205005  1
2810360 1
212420  1
1508114 2
5103865 2
15700484 2
9705414 2
210473  3
709925  3
5106485 4
514157  4
709342  4
507356  7
means = 2.35

---- 7 Former-Smoker (smoking_habit = 1)
1501052 1
3812245 1
1505193 1
1509349 2
3408711 2
508048  3
415185  4
mean = 2

---- 3 Current-Smoker (smoking_habit = 2)
1306378 2 (Father20, Sister17)
1501042 2 (Father20, Uncle20)
1300885 5 (Uncle16, Uncle9, Sister1, Mother17, Grandmother1)
mean = 3

SELECT DISTINCT t1.EHR as EHR, t6.smoking_habit as smoker, t8.cancer_type_translated as cancer, t7.familyMember_Translated as family
FROM `alk_mutation` t1, `PatientAge` t2, `family_antecedents` t3,  `family_antecedents_treatment_line` t4, `patient` t5, `smoker_information` t6, `familyRelationshipsENG` t7, `family_antecedents_treatment_line_translated` t8
where t1.EHR = t2.EHR and t1.EHR = t3.EHR and t1.EHR = t4.ehr and t1.EHR = t5.EHR and t1.EHR = t6.EHR and t4.family_member = t7.familyMember and t4.cancer_type = t8.cancer_type
and t2.Age <= 50 and t5.gender = 1 and (`result_alk_ihq` = '1.0' or `result_alk_fish` = '1.0' or `result_alk_rna` = '1.0') and t3.has_family_cancer_antecedents != -1 and t4.cancer_type not in (-1,888) and t6.smoking_habit not in (3,-1)
order by smoker, EHR

"EHR","smoker","cancer","family"
"1503660","0","Colorrectal   ","Uncle"
"1508114","0","Colorrectal   ","Father"
"1508114","0","Uterus/cervical  ","Mother"
"1508901","0","Lung  ","Mother"
"15700484","0","Esophagogastric   ","Mother"
"15700484","0","Melanoma  ","Father"
"203512","0","Lung  ","Grandmother"
"203552","0","Esophagogastric   ","Sister"
"205005","0","Esophagogastric   ","Grandfather"
"210473","0","Leukemia  ","Female_Cousin"
"210473","0","Breast  ","Aunt"
"210473","0","Breast  ","Female_Cousin"
"212420","0","Breast  ","Aunt"
"2810360","0","Colorrectal   ","Uncle"
"507356","0","Breast  ","Sister"
"507356","0","Lymphoma  ","Sister"
"507356","0","Central nervous system  ","Grandfather"
"507356","0","Colorrectal   ","Father"
"507356","0","Gall bladder  ","Mother"
"507356","0","Breast  ","Female_Cousin"
"507356","0","Colorrectal   ","Aunt"
"5103865","0","Lung  ","Uncle"
"5103865","0","Esophagogastric   ","Grandmother"
"5106485","0","Breast  ","Female_Cousin"
"5106485","0","Ovarian  ","Sister"
"5106485","0","Lung  ","Uncle"
"5106485","0","Breast  ","Aunt"
"514157","0","Central nervous system  ","Male_Cousin"
"514157","0","Esophagogastric   ","Male_Cousin"
"514157","0","Breast  ","Aunt"
"514157","0","Pancreatic   ","Aunt"
"709342","0","Colorrectal   ","Uncle"
"709342","0","Uterus/cervical  ","Grandmother"
"709342","0","Colorrectal   ","Aunt"
"709342","0","Skin no melanoma  ","Father"
"709925","0","Uterus/cervical  ","Grandmother"
"709925","0","Colorrectal   ","Female_Cousin"
"709925","0","Skin no melanoma  ","Father"
"9705414","0","Colorrectal   ","Father"
"9705414","0","Breast  ","Aunt"
"1501052","1","Lung  ","Father"
"1505193","1","Lung  ","Grandmother"
"1509349","1","Uterus/cervical  ","Aunt"
"1509349","1","Breast  ","Aunt"
"3408711","1","Colorrectal   ","Father"
"3408711","1","Esophagogastric   ","Uncle"
"3812245","1","Uterus/cervical  ","Mother"
"415185","1","Breast  ","Aunt"
"415185","1","Leukemia  ","Grandfather"
"415185","1","Lung  ","Uncle"
"508048","1","Unknown origin carcinoma  ","Grandmother"
"508048","1","Breast  ","Mother"
"508048","1","Lung  ","Uncle"
"508048","1","Lung  ","Father"
"1300885","2","Prostate  ","Uncle"
"1300885","2","Breast  ","Grandmother"
"1300885","2","Uterus/cervical  ","Mother"
"1300885","2","Breast  ","Sister"
"1300885","2","Pancreatic   ","Uncle"
"1306378","2","Uterus/cervical  ","Sister"
"1306378","2","Lung  ","Father"
"1501042","2","Lung  ","Uncle"
"1501042","2","Lung  ","Father"



SyntaxError: invalid syntax (1412101930.py, line 1)